In [ ]:
import sys
sys.path.insert(0, '../src')
import numpy as np
import pandas as pd
# from tensorflow.keras.preprocessing.
import matplotlib.pyplot as plt
import os
import cv2
import re
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from collections import namedtuple
import lmdb
from dataloader_iam import *
from preprocessor import *
from model import *
from main import *

In [ ]:
base_path = "/Users/nicolacave/dsi_galvanize/capstones/capstone3/handwriting_reader/data/img"
word_folders = [i for i in os.listdir(base_path) if not i == '.DS_Store']
# word_folders.remove('.DS_Store')
print(f"num folders: {len(word_folders)}")
img_paths = []

# print(f"base_path: {base_path}")

# Populate list of image filepaths

for folder in word_folders:  
    path = os.path.join(base_path,folder)
    word_subfolders = [i for i in os.listdir(path) if not i == '.DS_Store']
    # print(f"folder: {folder}")
    # print(f"path: {path}") 
    for subfolder in word_subfolders:
        newpath = os.path.join(path, subfolder)
        # print(f"subfolder: {subfolder}")
        # print(f"newpath: {newpath}")
        for img in os.listdir(newpath):
            # print(f"img: {img}")
            # print(path+"/"+folder+"/"+img)
            img_path = newpath + "/" + img
            img_paths.append(img_path)
            
print(f"num images: {len(img_paths)}")
# print(img_paths[444])

In [ ]:
img_arr = np.array(img_paths)
img_arr.shape

In [ ]:
# Get corresponding text labels

data_dir = "/Users/nicolacave/dsi_galvanize/capstones/capstone3/handwriting_reader/data"
f = open(data_dir + "/" + 'gt/words.txt')
chars = set()
bad_samples_reference = ['a01-117-05-02', 'r06-022-03-05']  # known broken images in IAM dataset
file_names = []
img_labels = []
for line in f:
    # ignore comment line
    if not line or line[0] == '#':
        continue

    line_split = line.strip().split(' ')
    assert len(line_split) >= 9
    img_labels.append(line_split[8])
    # filename: part1-part2-part3 --> part1/part1-part2/part1-part2-part3.png
    file_name_split = line_split[0].split('-')
    file_name_subdir1 = file_name_split[0]
    file_name_subdir2 = f'{file_name_split[0]}-{file_name_split[1]}'
    file_base_name = line_split[0] + '.png'
    file_name = data_dir + "/" + 'img' + "/" + file_name_subdir1 + "/" + file_name_subdir2 + "/" + file_base_name
    file_names.append(file_name)
    #print(file_name)
    if line_split[0] in bad_samples_reference:
        print('Ignoring known broken image:', file_name)
        continue

    # GT text are columns starting at 9
    gt_text = ' '.join(line_split[8:])
    chars = chars.union(set(list(gt_text)))
    
    # put sample into list
    samples = []
    samples.append(Sample(gt_text, file_name))
print(len(samples))
print(len(file_names))
print(len(img_labels))
# print(chars)
# print(img_labels[444])

In [ ]:
# Create table of X and y

file_names = pd.Series(file_names)
print(len(file_names))
img_labels = pd.Series(img_labels)
df = pd.DataFrame(list(zip(file_names, img_labels)), columns=["file_name", "img_label"])

# read and resize images

images = []
for i in file_names:
    curImg = cv2.imread(i)
    print(curImg)
    try:
        curImg = cv2.resize(curImg, (1400, 1000), interpolation=cv2.INTER_AREA)
        print(curImg.shape)
    except:
        break

height, width , layers = img.shape
size=(width,height)
print(size)
curImg = cv2.resize(curImg, (32,32))
images.append(curImg)

In [ ]:
# Create training set at 80%
# Create testing set at 20%

X_train, X_test, y_train, y_test = train_test_split(np.array(df["file_name"]), np.array(df["img_label"]), test_size=0.2)

In [ ]:
# print(train_words)
# print(train_samples)

In [ ]:
# DATADIR = "/Users/nicolacave/dsi_galvanize/capstones/capstone3/handwriting_reader/data/img"

# CATEGORIES = [i for i in os.listdir(DATADIR) if not i == ".DS_Store"]
# print(CATEGORIES)

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    #print(path)
    #if os.path.isdir(path):
    for folder in os.listdir(path):
        #print(f"folder: {folder}")
        for img in os.listdir(path+"/"+folder):
            #print(f"img: {img}")
            #print(path+"/"+folder+"/"+img)
            #if not img.startswith('.'):
            img_array = cv2.imread(os.path.join(path,folder,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                #print(img_array)
            #plt.imshow(img_array, cmap='gray')  # graph it
            #plt.show()  # display!

        #break  # we just want one for now so break
    #break  #...and one more!

In [ ]:
training_data = []
#print(len(CATEGORIES))
def create_training_data():
    print(len(CATEGORIES)*0.8)
    for category in CATEGORIES[:int(len(CATEGORIES)*0.8)]:  # do dogs and cats
        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
        for folder in os.listdir(path):
            for img in os.listdir(path+"/"+folder):
        #for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
                try:
                    img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array
                    new_array = cv2.resize(img_array, (128, 128), interpolation=cv2.INTER_AREA)  # resize to normalize data size
                    training_data.append([new_array, class_num])  # add this to our training_data
                except Exception as e:  # in the interest in keeping the output clean...
                    print(e)
                    #pass
                #except OSError as e:
                #    print("OSErrroBad img most likely", e, os.path.join(path,img))
                #except Exception as e:
                #    print("general exception", e, os.path.join(path,img))

create_training_data()


print(len(training_data))

In [ ]:
print(len(training_data))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",input_shape=[137,236,1]),#input shape: [rows, columns, channels]
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(training_data)